To put the model to the test, we use speech recognition to assist blinds in obtaining currency recognition information provided by the model. 

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.initializers import glorot_uniform
import speech_recognition as sr
from gtts import gTTS
import playsound
import os
import random

In [ ]:
model_filename = 'Tunisian_currency_recognition_model.h5'

In [ ]:
def alexis_speak(audio_string):
    tts = gTTS(text= audio_string,lang='en')
    r = random.randint(0,100000000)
    audio_file = 'audio-'+str(r)+'.mp3'
    tts.save(audio_file)
    playsound.playsound(audio_file)
    os.remove(audio_file)

In [ ]:
model=load_model(model_filename,custom_objects={'GlorotUniform': glorot_uniform()},compile=False)

In [ ]:
labels = ["10Dinars","20Dinars"]

You can try this code with your computer camera after downloading it. 

In [ ]:
# Here,we use the computer's embedded camera.

cap=cv2.VideoCapture(0)
r = sr.Recognizer()
while True:
    with sr.Microphone() as source:
        alexis_speak("how can I help you")
        audio = r.listen(source)
        print("listen")
        try:
            text = r.recognize_google(audio)
            print(text)
            if (text == "stop"):
                break
            else:
                alexis_speak("You said"+format(text))
                if (text == "what do you see"):
                     alexis_speak("Wait for second")
                     while True:
                         __,frame=cap.read()
                         gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                         gray=cv2.resize(gray,(80,80))
                         gray=np.array(gray)/255.0
                         img_vector = np.empty((0,))
                         for i in range(len(gray)):    
                             img_vector = np.r_[img_vector,gray[i,:]]
                         img_vector = img_vector.reshape(1,-1)
                         prob = model.predict(img_vector)[0]
                         index=np.argmax(prob)
                         probability=prob[index]*100
                         classe=labels[index]
                         text="{} : {}".format(classe,probability)
                         print(text)
                         alexis_speak(classe)
                         cv2.imshow("Tunisian_currency_recognition",frame)
                         wait=cv2.waitKey(1)
                         if wait==ord("q"):
                             break
                  
        except:
             alexis_speak("Sorry could not recognize what you said")
  
cap.release()  
cv2.destroyAllWindows()